### Models e Banco de Dados

| [Anterior](4.Estrutura-Diretorios.ipynb)| [Próximo](6.Rotas-e-Paginas.ipynb)     | 
| :------------- | :----------:|

### 1. Exemplo "Modelo" simulando um twitter

![image info](./images/posts_twitter_model.png)

### 2. Flask e SQLAlchemy (ORM)

Site: https://flask-sqlalchemy.palletsprojects.com/en/2.x/

* **Instalação** Saiba mais em [Flask-SQLAlchemy pip install](https://pypi.org/project/Flask-SQLAlchemy/)

> $ pip install -U Flask-SQLAlchemy

**OBS:** Lembre-se de instalá-lo no ambiente virtual (venv) do seu projeto.

### 3. Configurações iniciais do SQLAlchemy

* Imports e Declarações iniciais

```
import os.path

from flask import Flask
from flask_sqlalchemy import SQLAlchemy # (Import principal do SQLAlchemy)

basedir = os.path.abspath(os.path.dirname(__file__))

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(basedir, 'posts_twitter.db') # (Banco de Dados SQLite)
db = SQLAlchemy(app) # (Criando uma instância padrão db)
```

* A configuração acima deve ser declarada no arquivo "app/__ init __.py"

### 4. Trabalhando com Models no SQLAlchemy (arquivo tables.py)

```
from app import db

class User(db.Model):
    _tablename_ = "users"
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String, unique=True)
    password = db.Column(db.String(20))
    name = db.Column(db.String(300))
    email = db.Column(db.String, unique=True, nullable=False)

    def __init__(self, username, password, name, email):
        self.username = username
        self.password = password
        self.name = name
        self.email = email

    # Representação (representation) - Define como o User será representado
    def __repr__(self):
        return '<User %r>' % self.username
```

### 5. Migrações no Bancos de Dados

* Módulo Flask-Migrate: https://flask-migrate.readthedocs.io/en/latest/
> **Instalando:** $ pip install Flask-Migrate

* Módulo Flask-Script: https://flask-script.readthedocs.io/en/latest/
> Fornece suporte para escrever scripts externos no Flask. <br/>
> **Instalando:** $ pip install Flask-Script

* Realizando os **imports** do Flask-Migrate e Flask-Script no arquivo 'app/__ init __.py'

> from flask_migrate import Migrate <br />
> from flask_script import Manager, MigrateCommand <== Não funcionou

* Instanciando o Script e o Migrate

```
...
migrate = Migrate(app, db) # Instancia o Migrate

# O Manager se preocupa com os comandos de inicialização
manager = Manager(app) <== Não funcionou
manager.add_command('db', MigrateCommand) <== Não funcionou
...
```

* Com o Manager instanciado o arquivo "run.py" poderá  ser alterado 

Alterando a forma de execução da aplicação para adicionar comandos a ela:

> from app import manager <br/><br/>
if __ name __ == "__ main __":    
......manager.run()

* Criando o banco de dados e as respectivas tabelas somente quando o app estiver pronto

> if __ name __ == "__ main __":   <br/>
......db.init_app(app=app) <br/>
............with app.test_request_context(): <br/>
..................db.create_all() <br/>
......app.run()

* Comandos diversos - após configuração acima

```
$ flask db init # Inicia o arquivo SQLite em ...
.../flask-api/migrations/<arquivos de migração e tabela alembic>

$ flask db migrate -m "Initial migration."

$ flask db upgrade 

- Com isso, será criado o arquivo "posts_twitter" na raiz do projeto.
```

### 6. Configurações relacionadas ao Banco de Dados

* Arquivo de Configuração `config.py` na raiz do projeto

> DEBUG = True # Enquanto estiver desenvolvendo - ativa a atualização e compilação automática

> SQLALCHEMY_DATABASE_URI = 'sqlite:///posts_twitter.db' <br/>
> SQLALCHEMY_TRACK_MODIFICATIONS = True

* Importanto o arquivo `config.py` dentro do `app/__ init __.py`

> ```app.config.from_object('config')```